In [286]:
import re
import numpy as np
import pandas as pd
import itertools
from collections import Counter
from tqdm.notebook import trange

Данная задача является np-полной. Существует несколько подходов к решению данной задачи, обладающих различными нюансами. Мой вариант решения будет опираться на случайный перебор, это позволит найти глобальный максимум функции, но потребует значительных временных затрат на исполнение всех итераций.

In [238]:
text = '''Во глубине сибирских руд
Храните гордое терпенье,
Не пропадет ваш скорбный труд
И дум высокое стремленье.'''


# Функция для получения очищенного текста без пробелов между словами.
def set_letters(text):
    s = re.sub(r'[^а-яА-ЯёЁ]', ' ', text).lower()
    return ''.join(s.split())


# Функция для получения очищенного текста.
def clear_text(text):
    s = re.sub(r'[^а-яА-ЯёЁ]', ' ', text).lower()
    return ' '.join(s.split())


set_1 = set_letters(text)
clear_1 = clear_text(text)
all_letters_dict = Counter(set_1)
all_letters = pd.DataFrame(index=all_letters_dict.keys(),
                           data={'count': list(all_letters_dict.values()),
                                 'freq': [round(i / len(set_1) * 100, 2) for i in all_letters_dict.values()]})

# Список словарей по словам
letters_in_words = [Counter(i) for i in clear_1.split()]

# Максимальное количество букв, встречающихся в одном слове
max_letters_in_words = {k: v for d in letters_in_words for k, v in d.items()}

# Для ускорения работы алгоритма уберем два значения – буквы 'ш' и 'й', как буквы с наименьшим вероятностным ожиданием - данные буквы встречаются лишь по одному разу во всем множестве букв.
# Будет логичным дозаказывать данные буквы.
for i in all_letters.query('count == 1').index:
    del max_letters_in_words[i]
max_letters_in_words

{'в': 1,
 'о': 2,
 'г': 1,
 'л': 1,
 'у': 1,
 'б': 1,
 'и': 1,
 'н': 1,
 'е': 3,
 'с': 1,
 'р': 1,
 'к': 1,
 'х': 1,
 'д': 1,
 'а': 1,
 'т': 1,
 'п': 2,
 'ь': 1,
 'ы': 1,
 'м': 1}

Буквы, которые во всем множестве встречаются только один раз.

In [254]:
all_letters.query('count == 1').index

Index(['ш', 'й'], dtype='object')

Вероятность встретить какую-либо из присутствующих букв во всем корпусе. А также общее количество каждой из букв во всем корпусе.

In [255]:
all_letters.sort_values(by='count')

,count,freq
ш,1,1.14
й,1,1.14
м,2,2.27
ы,2,2.27
г,2,2.27
л,2,2.27
ь,2,2.27
х,2,2.27
п,3,3.41
а,3,3.41


Найдем диапазон значений для каждой из переменных, от 0 до max_count, которое можно встретить в одном слове.

In [289]:
variations = {}
for letter, frequency in max_letters_in_words.items():
    variations[letter] = list(range(frequency + 1))

variations

{'в': [0, 1],
 'о': [0, 1, 2],
 'г': [0, 1],
 'л': [0, 1],
 'у': [0, 1],
 'б': [0, 1],
 'и': [0, 1],
 'н': [0, 1],
 'е': [0, 1, 2, 3],
 'с': [0, 1],
 'р': [0, 1],
 'к': [0, 1],
 'х': [0, 1],
 'д': [0, 1],
 'а': [0, 1],
 'т': [0, 1],
 'п': [0, 1, 2],
 'ь': [0, 1],
 'ы': [0, 1],
 'м': [0, 1]}

Для уменьшения количества итераций подбора итогового решения уберем значения с крайне малой вероятностью. Для этого обратимся к ранее полученной таблице частот.

In [293]:
variations = {'в': [0, 1],
              'о': [1, 2],
              'г': [0, 1],
              'л': [0, 1],
              'у': [0, 1],
              'б': [0, 1],
              'и': [1],
              'н': [1],
              'е': [1, 2, 3],
              'с': [1],
              'р': [1],
              'к': [0, 1],
              'х': [0, 1],
              'д': [1],
              'а': [0, 1],
              'т': [1],
              'п': [0, 1, 2],
              'ь': [0, 1],
              'ы': [0, 1],
              'м': [0, 1]}

keys = variations.keys()
value_combinations = list(itertools.product(*[variations[key] for key in keys]))

result_list = []

for values in value_combinations:
    new_dict = max_letters_in_words.copy()
    for key, value in zip(keys, values):
        new_dict[key] = value
    result_list.append(new_dict)

Итоговая длина списка словарей.

In [294]:
len(result_list)

36864

In [301]:
# Зафиксируем псевдослучайность
np.random.seed(42)
best_amount = 0
best_dict = None

for check_dict in trange(len(result_list)):
    amount_day = 0

    for _ in range(1000):
        amount_day -= sum(result_list[check_dict].values())
        discrepancy_counter = 0
        control_dict = Counter(clear_1.split()[np.random.randint(0, len(clear_1.split()))])

        for key, value in control_dict.items():
            if key not in result_list[check_dict] or result_list[check_dict][key] < value:
                discrepancy_counter += 1

        if discrepancy_counter > 1:
            amount_day -= 10
        elif discrepancy_counter == 1:
            amount_day += 38
        else:
            amount_day += 40

    if best_amount < amount_day:
        best_amount = amount_day
        best_dict = result_list[check_dict]

  0%|          | 0/36864 [00:00<?, ?it/s]

Подобранное решение было получено при проверке каждого словаря на 100000 случайно выбранных слов из заданного корпуса.

In [302]:
best_dict

{'в': 1,
 'о': 1,
 'г': 1,
 'л': 1,
 'у': 0,
 'б': 1,
 'и': 1,
 'н': 1,
 'е': 1,
 'с': 1,
 'р': 1,
 'к': 1,
 'х': 0,
 'д': 1,
 'а': 1,
 'т': 1,
 'п': 1,
 'ь': 1,
 'ы': 1,
 'м': 1}

In [305]:
best_amount / 1000

17.432